In [10]:
import os

In [17]:
%pwd

'/Users/sushanth/Desktop/MLOps/Yoga_Pose_Image_Classification/research'

In [18]:
os.chdir("../")

In [19]:
%pwd

'/Users/sushanth/Desktop/MLOps/Yoga_Pose_Image_Classification'

In [20]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/sushanth-0/Yoga_Pose_Image_Classification.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="sushanth-0"
os.environ["MLFLOW_TRACKING_PASSWORD"]="1df6ac3360167070b263f4f63b1a6628069a2946"

In [42]:
import tensorflow as tf

In [43]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [44]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: tuple
    params_batch_size: int

In [45]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [46]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Yoga dataset",
            mlflow_uri="https://dagshub.com/sushanth-0/Yoga_Pose_Image_Classification.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config


In [47]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [48]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.models.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        # Workaround to set keras version
        if not hasattr(tf.keras, '__version__'):
            tf.keras.__version__ = tf.__version__

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({"loss": self.score[0], "accuracy": self.score[1]})

            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")


In [49]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-06-13 21:32:11,966: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-06-13 21:32:11,978: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-13 21:32:11,994: INFO: common: created directory at: artifacts]
Found 1749 images belonging to 107 classes.


2024-06-13 21:32:12.598094: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


55/55 [==============================] - 13s 226ms/step - loss: 28.6799 - accuracy: 0.0938
[2024-06-13 21:32:25,479: INFO: common: json file saved at: scores.json]


2024/06/13 21:32:25 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /var/folders/58/fc1l05pd7zb81y2wfqc56s480000gn/T/tmpag2lcf8p/model/data/model/assets
[2024-06-13 21:32:26,638: INFO: builder_impl: Assets written to: /var/folders/58/fc1l05pd7zb81y2wfqc56s480000gn/T/tmpag2lcf8p/model/data/model/assets]


2024/06/13 21:32:29 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /var/folders/58/fc1l05pd7zb81y2wfqc56s480000gn/T/tmpag2lcf8p/model, flavor: tensorflow), fall back to return ['tensorflow==2.13.0']. Set logging level to DEBUG to see the full traceback.
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/06/13 21:32:47 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 5
Created version '5' of model 'VGG16Model'.
